<a href="https://colab.research.google.com/github/Ava-Jia/Ava-Jia/blob/main/LangGraph-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [17]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"

# Making the GraphState

In [3]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

# 定义了一个字典的结构，明确指定这几个键以及它们值的类型
# AgentState存储了代理在整个对话流程中需要维护的所有信息。它在节点之间传递，并被节点修改。
# 充当了代理的记忆，允许它跟踪对话历史、推理过程和采取的行动
# 包括用户输入、对话历史记录、代理输出、中间步骤
class AgentState(TypedDict):
  # The Input String
  input: str
  # The list of previous messages in the conversation
  chat_history: list[BaseMessage]
  # 代理的输出（例如，采取的动作或完成的标志）
  # Union类型：允许你指定一个变量可以接受多种类型的值
  # Needs `None` as a valid type, since this is what this will start as
  agent_outcome: Union[AgentAction, AgentFinish, None]
  # List of actions and corresponding observations
  # Here we annotate this with `operator.add` to indicate that operations to
  # this state should be ADDED to the existing values (not overwrite it)
  # Annotated类型: 允许你将类型信息与元数据（metadata）关联起来
  intermediate_steps: Annotated[List[tuple[AgentAction, str]], operator.add]

# Custom Tools
Tools are interfaces that an agent can use t interact with the world. Such as:
1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
Whether the result of a tool should be returned directly to the user

In [5]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [6]:
# 两个自定义的LangChain工具
import random

# 装饰器：将该函数注册为一个 LangChain 工具
# return_direct=True：工具的返回值应该直接返回给用户，而不是继续交给 LLM 进行处理
@tool("lower_case", return_direct=True)
# 它接收一个字符串类型的 input 作为参数，并返回一个字符串
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

# 接受一个字符串输入，但实际上并没有使用它。这可能是为了与其他工具保持一致的接口
@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
  """Returns a random number between 0-100."""
  return random.randint(0, 100)

tools = [to_lower_case, random_number_maker]

In [7]:
to_lower_case.run("Make Me A CAKE")

'make me a cake'

In [11]:
random_number_maker.run('1')

36

# Agent - with new create_openai

In [13]:
from langchain import hub #用于从 LangChain Hub 获取预定义的提示词（prompts）
from langchain.agents import create_openai_functions_agent
# 用于使用 OpenAI Functions 创建一个 LangChain 代理
from langchain_openai.chat_models import ChatOpenAI

#  从 LangChain Hub 下载一个预定义的提示词。
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the llm model
# 流式输出: 模型会逐步返回结果，而不是等待所有结果都生成完毕才一次性返回。
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt=prompt)

In [14]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [18]:
inputs = {"input":"give me a random number and then write in words and make it lower case",
      "chat_history":[],
      "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-NVZ4F***************************************qwUn. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
agent_outcome

In [ ]:
type(agent_runnable)

# Nodes

In [23]:
!pip uninstall langgraph
!pip install langgraph

Found existing installation: langgraph 0.3.24
Uninstalling langgraph-0.3.24:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/langgraph-0.3.24.dist-info/*
    /usr/local/lib/python3.11/dist-packages/langgraph/_api/*
    /usr/local/lib/python3.11/dist-packages/langgraph/channels/*
    /usr/local/lib/python3.11/dist-packages/langgraph/config.py
    /usr/local/lib/python3.11/dist-packages/langgraph/constants.py
    /usr/local/lib/python3.11/dist-packages/langgraph/errors.py
    /usr/local/lib/python3.11/dist-packages/langgraph/func/*
    /usr/local/lib/python3.11/dist-packages/langgraph/graph/*
    /usr/local/lib/python3.11/dist-packages/langgraph/managed/*
    /usr/local/lib/python3.11/dist-packages/langgraph/pregel/*
    /usr/local/lib/python3.11/dist-packages/langgraph/py.typed
    /usr/local/lib/python3.11/dist-packages/langgraph/types.py
    /usr/local/lib/python3.11/dist-packages/langgraph/utils/*
    /usr/local/lib/python3.11/dist-packages/langgraph/version.py
Proceed (Y

In [25]:
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.tools import BaseTool

def execute_tool_1(action: AgentAction, tools: list[BaseTool]):
    """Executes a tool and returns the result."""
    tool_name = action.tool
    tool_input = action.tool_input

    for tool in tools:
        if tool.name == tool_name:
            return tool.run(tool_input)

    raise ValueError(f"Tool {tool_name} not found.")

In [27]:
# Define the agent/graph
# 接收一个 data 字典作为输入，并将该字典传递给 agent_runnable.invoke()
# 代理的输出被赋值给 agent_outcome，然后存储在新的字典 {"agent_outcome": agent_outcome} 中并返回
def run_agent(data):
  agent_outcome = agent_runnable.invoke(data)
  return {"agent_outcome": agent_outcome}

# Define the function to execute tools
# 将 agent_action 和 output 封装成一个元组，并将该元组添加到 intermediate_steps, 存储在新的字典中并返回
def execute_tools(data):
  # Get the most recent agent_outcome - this is the key added in the `agent` above
  agent_action = data['agent_outcome']
  # Execute the tool
  output = execute_tool_1(agent_action, tools)
  print(f"The agent action is: {agent_action}")
  print(f"The output of the tool is: {output}")

  return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
# 决定是否继续执行代理循环
def should_continue(data):
  # If the agent outcome is an AgentFinish, the we return `exit` string
  if isinstance(data['agent_outcome'], AgentFinish):
    return "exit"
  # Otherwise, we return `continue`
  return "continue"

# Define the Graph

In [30]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
# 向图中添加了一个名为 "agent" 的节点，并将 run_agent 函数作为该节点的执行逻辑。
# run_agent 函数应该接收一个 AgentState 对象作为输入，并返回一个更新后的 AgentState 对象。
workflow.add_node("agent", run_agent)
# 向图中添加了一个名为 "action" 的节点，并将 execute_tools 函数作为该节点的执行逻辑。
# execute_tools 函数应该接收一个 AgentState 对象作为输入，并返回一个更新后的 AgentState 对象。
workflow.add_node("action", execute_tools)

# Set the entrypoint as a `agent`
# 将 "agent" 节点设置为图的入口点。这意味着当图开始执行时，会首先调用 "agent" 节点。
workflow.set_entry_point("agent")

# Now Add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node.We use 'agent'.
    # This means these are the edges taken after the 'agent' node is called.
    "agent",
    # Next, we pass in the function that wil determine which node is called next.
    should_continue,
    # Finally we pass in a mapping
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # what will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `continue`, them we call the tool node.
        "continue": "action",
        # If `exit`, finish.
        "exit": END
    }

)

# Add a nornal edge from `tools` to `agent`
workflow.add_edge("action", "agent")

# This compile it into a LangChain Runnable
app = workflow.compile()

In [31]:
workflow.branches

defaultdict(dict,
            {'agent': {'should_continue': Branch(path=should_continue(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), ends={'continue': 'action', 'exit': '__end__'}, then=None, input_schema=None)}})

In [32]:
workflow.nodes, workflow.edges

({'agent': StateNodeSpec(runnable=agent(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, ends=()),
  'action': StateNodeSpec(runnable=action(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None, ends=())},
 {('__start__', 'agent'), ('action', 'agent')})

In [33]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-NVZ4F***************************************qwUn. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

output = app.invoke(inputs)

In [ ]:
output.get("agent_outcome").return_values['output']

In [ ]:
output.get("intermediate_steps")